# Building RAG Chatbots for Buissness with the help of LangChain, Open AI API, Pinecone DB

In [1]:
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "Replace with your OpenAI API Key"

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful bussiness assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I am doing great. How can I help you today?")
    ]

In [6]:
res=chat(messages)
res

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Hello! How can I assist you today?')

Let us ask a question on Yardstick AI before training the model

In [7]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you tell me about the Yardstick AI company?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

I'm sorry, but I don't have specific information about a company named Yardstick AI. If you provide me with more details or context about the company, I can try to assist you further. Alternatively, you can search for information about Yardstick AI online to learn more about their products, services, and offerings.


It failed to fetch the details of the company as it has  knowledge limited to that of 2021

So, firstly we will train the model on Yardstick AI and then on it will act as an Buissness Assistant, answering the relevent doubt for the user.
We do so by retriving the contents from it's website

### Importing the Data

In [8]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

# Define root domain to crawl
domain = "www.yardstick.live"
full_url = "https://www.yardstick.live/"

# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# Function to get the hyperlinks from a URL
def get_hyperlinks(url):

    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []

            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))


def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w") as f:

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # Get the text but remove the tags
            text = soup.get_text()

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")

            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://www.yardstick.live/
https://www.yardstick.live/#contact
https://www.yardstick.live
https://www.yardstick.live/#about
https://www.yardstick.live/#projects
https://www.yardstick.live/blog
https://www.yardstick.live/blog/Latest-news-articles-in-AI
https://www.yardstick.live/blog/Latest-news-articles-in-AI/Competition-in-AI-Video-Generation-Heats-Up-as-DeepMind-Alums-Unveil-Haiper
https://www.yardstick.live/blog/Latest-news-articles-in-AI/Ema-a-Universal-AI-Employee-Emerges-from-Stealth-with-25M
https://www.yardstick.live/blog/Latest-news-articles-in-AI/The-Betrayal-of-a-Vision-Elon-Musk's-Legal-Battle-with-OpenAI-and-Sam-Altman
https://www.yardstick.live/blog/Latest-news-articles-in-AI/India's-New-AI-Regulation-Sparks-Controversy-A-Deep-Dive-into-the-Reversal
https://www.yardstick.live/#home
https://www.yardstick.live/project/ayodhya-gpt
https://www.yardstick.live/tel://+917891053001
HTTP Error 308: Permanent Redirect


In [9]:
def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

In [10]:
import pandas as pd

# Create a list to store the text files
texts=[]

# Get all the text files in the text directory
for file in os.listdir("text/" +domain + "/"):

    # Open the file and read the text
    with open("text/" + domain + "/" + file, "r") as f:
        text = f.read()

        # Omit the first 11 lines and the last 4 lines, then replace -, _, and #update with spaces.
        texts.append((file[11:-4].replace('-',' ').replace('_', ' ').replace('#update',''), text))

# Create a dataframe from the list of texts
df = pd.DataFrame(texts, columns = ['fname', 'text'])

# Set the text column to be the raw text with the newlines removed
df['text'] = df.fname + ". " + remove_newlines(df.text)
df.to_csv('processed/scraped.csv')
df.head()


<ipython-input-9-429202555403>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  serie = serie.str.replace('\\n', ' ')


,fname,text
0,ck.live #projects,ck.live #projects. Yardstick AIYardstick AIHom...
1,ck.live project ayodhya gpt,ck.live project ayodhya gpt. Blog Details Page...
2,ck.live blog Latest news articles in AI The Be...,ck.live blog Latest news articles in AI The Be...
3,ck.live blog,ck.live blog. BlogYardstick AIHomeAboutProject...
4,ck.live,ck.live. Yardstick AIYardstick AIHomeAboutProj...


In [11]:
df.iloc[:,:]

,fname,text
0,ck.live #projects,ck.live #projects. Yardstick AIYardstick AIHom...
1,ck.live project ayodhya gpt,ck.live project ayodhya gpt. Blog Details Page...
2,ck.live blog Latest news articles in AI The Be...,ck.live blog Latest news articles in AI The Be...
3,ck.live blog,ck.live blog. BlogYardstick AIHomeAboutProject...
4,ck.live,ck.live. Yardstick AIYardstick AIHomeAboutProj...
5,ck.live,ck.live . Yardstick AIYardstick AIHomeAboutPro...
6,ck.live blog Latest news articles in AI India'...,ck.live blog Latest news articles in AI India'...
7,ck.live #contact,ck.live #contact. Yardstick AIYardstick AIHome...
8,ck.live tel: +917891053001,ck.live tel: +917891053001. 404 | Not FoundYa...
9,ck.live #about,ck.live #about. Yardstick AIYardstick AIHomeAb...


### Building the Knowledge Base

We now have a dataset that can serve as our chatbot knowledge base. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

We begin by initializing our connection to Pinecone.

In [12]:
from pinecone import Pinecone

api_key = os.getenv("PINECONE_API_KEY") or "Repace With Your API Key"

pc = Pinecone(api_key=api_key)

I have already initialized an index in Pinecone DB in its offcial website.
Create an index in the [Pinecone](https://www.pinecone.io) cloud service.
Write the name of the index in the cell below. Also initialize the index with the cosine similarity index in Pinecone.

In [14]:
import time

index_name = 'Replace with your index name'


# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


Embedding the data to the Open AI embedding model(text-embedding-ada-002)

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

from tqdm.auto import tqdm  # for progress bar
batch_size = 1

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(len(df), i+batch_size)
    # get batch of data
    batch = df.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{i}" ]
    # get text to embed
    texts = [x['text'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['text'],
         'title': x['fname']} for i, x in batch.iterrows()
    ]

    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


  0%|          | 0/14 [00:00<?, ?it/s]

If there was huge data I would have increased the batch size. But since it is smaller(just 14 rows), I declared it to be 1

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00014,
 'namespaces': {'': {'vector_count': 14}},
 'total_vector_count': 14}

### Querying the data from the Vector DB

To use LangChain here we need to load the LangChain abstraction for a vector index, called a `vectorstore`. We pass in our vector `index` to initialize the object.
(I am querying using the cosine similarity measure).

In [17]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Using this `vectorstore` we can already query the index and see if we have any relevant information given our question about Yardstick AI.

In [18]:
query =" What is Yardstick AI? "

In [19]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [20]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    ck.live #about. Yardstick AIYardstick AIHomeAboutProjectAyodhaya GPTMagic SEOLive AI Sales CallChatBotBlogsContact UsWelcome to Tomorrow's Intelligence: Elevate with YardstickEmbark on a journey where the present meets the future—where AI brilliance unfolds effortlessly. At Yardstick, we're not just pioneers, we're architects of your AI excellence.Talk To UsExpert AI Services Tailored For Your BusinessWe leverage cutting-edge technology to create advanced frameworks that foster improved efficiency, better decision-making and redefine user interactions.AI IntegrationWe seamlessly integrate AI models into your existing systems to improve efficiency and propel business growth.Fine-TuningOur solutions are tailor-made to meet your specific needs and ensure optimal performance for your unique requirements.Advanced AnalyticsLeverage AI to gain valuable insights from your data and make better, data-driven decisions.Image Generation

There is a lot of text here, so let's pass it onto our chat model to see how it performs.

In [21]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

Yardstick AI is a company that heralds a new dawn in AI innovation. They have a skilled team and advanced AI solutions that are transforming the way businesses operate. They pride themselves on their commitment to excellence and dedication to pushing the frontiers of what AI can achieve. Yardstick provides a suite of AI services, including AI model integration, advanced analytics, image and video generation, and speech AI. Their agile solutions are tailored to the needs of their clients, ensuring optimal performance and strategic advantage. If you choose Yardstick, you will be part of a future where innovation meets excellence, and AI drives success.


Now, the model is trained so it is able to produce the relevent information about Yardstick AI

Let us ask another general question by a user who is using the service of Yardstick AI and has to get in touch with customer service of Yardstick AI.

In [22]:
query=" How to get in touch with the customer service of Yardstick AI?"

In [23]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

To get in touch with the customer service of Yardstick AI, you can send them a message through their contact form on their website. You can find the contact form on the "Contact Us" page of the Yardstick AI website. Simply fill in your name, email address, and requirements, and they will get back to you as soon as possible. You can also reach them via email at contact@yardstick.live or call them at +917891053001.


It is able to clearly provide the information to connect with the customer Support of Yardstick AI

In [ ]:
pc.delete_index(index_name)

---